In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import requests
import time

In [30]:
def fetch_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    proxy_list = []

    table_rows = soup.find('table' , class_='table table-striped table-bordered').find('tbody').find_all('tr')
    
    for row in table_rows:
        cols = row.find_all("td")
        ip = cols[0].text
        port = cols[1].text
        https = cols[6].text
        if https == "yes":
            proxy_list.append(f"{ip}:{port}")

    return proxy_list

In [31]:
def validate_proxies(proxies, test_url="https://httpbin.org/ip", timeout=5):
    working = []
    for proxy in proxies:
        try:
            res = requests.get(test_url, proxies={
                "http": f"http://{proxy}",
                "https": f"http://{proxy}"
            }, timeout=timeout)
            if res.status_code == 200:
                print(f"[✅] Working: {proxy}")
                working.append(proxy)
        except:
            print(f"[❌] Failed: {proxy}")
        time.sleep(1)
    return working

In [32]:
def scrape_with_rotation(url, proxy_list):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    for attempt in range(10):
        proxy = random.choice(proxy_list)
        print(f"Trying with proxy: {proxy}")
        try:
            res = requests.get(url, proxies={
                "http": f"http://{proxy}",
                "https": f"http://{proxy}"
            }, headers=headers, timeout=10)
            if res.status_code == 200:
                print(f"[✅] Success with {proxy}")
                return res.text
        except Exception as e:
            print(f"[⚠️] Failed with {proxy}: {e}")
    print("❌ All proxies failed.")
    return None

In [33]:
def lazy_scroll(driver):
    last_height = driver.execute_script('return document.body.scrollHeight')
    move_to = 3000

    while True:
        driver.execute_script(f'scrollTo(0,{move_to})')
        time.sleep(1)

        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == last_height:
            break

        last_height = new_height
        move_to += 3000

In [34]:
def aliexpress_page_scraper(html):

    soup = BeautifulSoup(html , 'html.parser')
    products = soup.find_all('div' , class_ = 'kr_k3')

    for product in products:

        product_title = product.find('div', class_ = 'kr_av').get('title')
        product_price = product.find('div', class_ = 'kr_kj').text.replace(',' , '')
        product_link  = product.find('a', class_ = 'kr_b in_is search-card-item').get('href')

        titles.append(product_title)
        prices.append(float(product_price[3:]))
        links.append(product_link)

In [35]:
def driver_init():
    options = Options()
    options.add_argument('--incognito')

    driver = webdriver.Chrome(options=options)
    return driver

In [36]:
titles = []
prices = []
links  = []

In [ ]:
page = 1

while True:

    url = f'https://www.aliexpress.com/w/wholesale-airpods.html?page={page}'
    driver = driver_init()
    driver.get(url)

    for _ in range(5):

        time.sleep(2)
        lazy_scroll(driver)
        aliexpress_page_scraper(driver.page_source)
        next_btn = driver.find_element(By.CLASS_NAME, 'comet-pagination-next')
        if next_btn.get_attribute('aria-disabled') == 'false':
            driver.execute_script("arguments[0].click();", next_btn)
        else:
            break

    if next_btn.get_attribute('aria-disabled') == 'false':
        page += 5
        driver.quit()
        continue
    else:
        break

WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=137.0.7151.69)
Stacktrace:
	GetHandleVerifier [0x0x7ff7d832fea5+79173]
	GetHandleVerifier [0x0x7ff7d832ff00+79264]
	(No symbol) [0x0x7ff7d80e9c8c]
	(No symbol) [0x0x7ff7d80d6f6b]
	(No symbol) [0x0x7ff7d80d5e2e]
	(No symbol) [0x0x7ff7d80fac44]
	(No symbol) [0x0x7ff7d81703c5]
	(No symbol) [0x0x7ff7d8190922]
	(No symbol) [0x0x7ff7d8168743]
	(No symbol) [0x0x7ff7d81314c1]
	(No symbol) [0x0x7ff7d8132253]
	GetHandleVerifier [0x0x7ff7d85fa2dd+3004797]
	GetHandleVerifier [0x0x7ff7d85f472d+2981325]
	GetHandleVerifier [0x0x7ff7d8613380+3107360]
	GetHandleVerifier [0x0x7ff7d834aa2e+188622]
	GetHandleVerifier [0x0x7ff7d83522bf+219487]
	GetHandleVerifier [0x0x7ff7d8338df4+115860]
	GetHandleVerifier [0x0x7ff7d8338fa9+116297]
	GetHandleVerifier [0x0x7ff7d831f558+11256]
	BaseThreadInitThunk [0x0x7ff879ac7374+20]
	RtlUserThreadStart [

In [ ]:
df = pd.DataFrame({
    'Titles' : titles,
    'Prices' : prices,
    'Links'  : links
})

df.shape

(1083, 3)

In [ ]:
df = df[df['Links'].str.contains('item')]
df.reset_index( drop=True , inplace=True)
df.index = range(1,len(df)+1)
df.shape

(996, 3)

In [ ]:
df.to_csv('aliexpress_products.csv')
df.head()

,Titles,Prices,Links
1,Super action hero underwear AirPods1/2/3 gener...,1552.15,//www.aliexpress.com/item/1005009147175343.htm...
2,Korean Shinny Mirror Chrome Acrylic Plastic Ca...,1072.01,//www.aliexpress.com/item/1005009179623144.htm...
3,for airpod 3 2021 case Silicone Soft Skin Prot...,641.40,//www.aliexpress.com/item/1005006438445350.htm...
4,Cover for AirPods Pro 2 Wireless Bluetooth Ear...,656.80,//www.aliexpress.com/item/1005006630268718.htm...
5,Earphones Case For AirPods Pro 1st 2nd generat...,998.66,//www.aliexpress.com/item/1005009088725490.htm...
